In [ ]:
import logging

import config
import lightgbm as lgb
import polars as pl
from catboost import CatBoostClassifier, CatBoostRegressor
from lifelines import BreslowFlemingHarringtonFitter, KaplanMeierFitter
from sklearn import linear_model

from src.customs.fold import add_kfold
from src.customs.metrics import CatBoostMetric, Metric, ROCAUCMetric
from src.feature.tabular import AggregateEncoder, OrdinalEncoder, RawEncoder
from src.model.sklearn_like import CatBoostClassifierWrapper, CatBoostRegressorWrapper, LightGBMWapper, LinearWrapper
from src.trainer.tabular.simple import single_inference_fn, single_train_fn

logger = logging.getLogger(__name__)
logging.basicConfig(level=logging.INFO)

logger = logging.getLogger(__name__)
logging.basicConfig(level=logging.INFO)

In [ ]:
class CreateTargetFn:
    def __init__(self, time_col: str, event_col: str):
        self.time_col = time_col
        self.event_col = event_col
        self.target_cols = [
            "target_kmf_race_group",
            "target_kmf",
            "target_xgboost_cox",
            "target_bfhf",
        ]

    def __call__(self, df: pl.DataFrame) -> pl.DataFrame:
        # add kmf target by race_group
        dfs = []
        for _, i_df in df.group_by("race_group", maintain_order=True):
            kmf = KaplanMeierFitter()
            kmf.fit(i_df[self.time_col], event_observed=i_df[self.event_col])
            _df = i_df.with_columns(
                pl.Series("target_kmf_race_group", kmf.survival_function_at_times(i_df[self.time_col])),
            )
            dfs.append(_df)
        target_df = pl.concat(dfs)

        # add kmf target
        kmf = KaplanMeierFitter()
        kmf.fit(target_df[self.time_col], event_observed=target_df[self.event_col])
        target_df = target_df.with_columns(
            pl.Series("target_kmf", kmf.survival_function_at_times(target_df[self.time_col])),
        )

        # add xgboost cox target
        target_df = target_df.with_columns(
            pl.when(target_df[self.event_col] == 0)
            .then(-1 * target_df[self.time_col])
            .otherwise(target_df[self.time_col])
            .alias("target_xgboost_cox"),
        )
        # add bfhf target
        bfhf = BreslowFlemingHarringtonFitter()
        bfhf.fit(target_df[self.time_col], event_observed=target_df[self.event_col])
        target_df = target_df.with_columns(
            pl.Series("target_bfhf", bfhf.survival_function_at_times(target_df[self.time_col])),
        )

        return target_df


def load_data():
    raw_train_df = pl.read_csv(config.COMP_DATASET_DIR / "train.csv").with_columns(
        pl.lit("TRAIN").alias(config.DATASET_COL),
        pl.lit(-1).alias(config.FOLD_COL),
    )

    create_target_fn = CreateTargetFn(time_col=config.SURVIVAL_TIME_COL, event_col=config.EVENT_COL)
    raw_train_df = create_target_fn(raw_train_df)

    raw_test_df = pl.read_csv(config.COMP_DATASET_DIR / "test.csv").with_columns(
        pl.lit("TEST").alias(config.DATASET_COL)
    )

    config.META_COLS = sorted(list(set(config.META_COLS + create_target_fn.target_cols)))

    train_test_df = pl.concat([raw_train_df, raw_test_df], how="diagonal_relaxed").sort(
        config.DATASET_COL, config.ID_COL, descending=[True, False]
    )
    return train_test_df


def load_ensemble_source(
    train_test_df: pl.DataFrame,
    exp_names: dict[str, str],
    source_prefix: str = "",
    output_dataset: str = "TRAIN",
) -> pl.DataFrame:
    te_result_path = (  # noqa
        lambda exp_name, model_name: config.ARTIFACT_EXP_DIR(exp_name) / model_name / "te_result.csv"
        if not config.IS_KAGGLE_ENV
        else config.OUTPUT_DIR / exp_name / model_name / "te_result.csv"
    )

    source_dfs = [
        pl.concat(
            [
                pl.read_csv(
                    config.ARTIFACT_EXP_DIR(exp_name) / model_name / "va_result.csv", columns=[config.ID_COL, "pred"]
                ).rename({"pred": f"pred_{exp_name}"}),
                pl.read_csv(te_result_path(exp_name, model_name), columns=[config.ID_COL, "pred"]).rename(
                    {"pred": f"pred_{exp_name}"}
                ),
            ],
            how="diagonal_relaxed",
        )
        for exp_name, model_name in exp_names.items()
    ]
    source_df = source_dfs.pop(0)
    for df in source_dfs:
        source_df = source_df.join(df, on=config.ID_COL, how="left")

    # add prefix
    source_df = source_df.select(
        [
            config.ID_COL,
            pl.all().exclude(config.ID_COL).name.prefix(source_prefix),
        ]
    )
    return train_test_df.join(source_df, on=config.ID_COL, how="left").filter(
        pl.col(config.DATASET_COL) == output_dataset
    )


def fe(
    train_test_df: pl.DataFrame,
    output_dataset: str = "TRAIN",
) -> pl.DataFrame:
    encoders = [
        RawEncoder(columns=config.META_COLS, prefix=""),
        RawEncoder(
            columns=(
                [
                    *config.NUMERICAL_COLS,
                ]
            ),
            prefix=f"{config.FEATURE_PREFIX}n_",
        ),
        OrdinalEncoder(
            columns=(
                [
                    *config.CATEGORICAL_COLS,
                ]
            ),
            prefix=f"{config.FEATURE_PREFIX}c_",
        ),
    ]

    for encoder in encoders:
        encoder.fit(train_test_df.filter(pl.col(config.DATASET_COL) == "TRAIN"))

    features_df = pl.concat(
        [
            encoder.transform(
                train_test_df.filter(pl.col(config.DATASET_COL) == output_dataset),
            )
            for encoder in encoders
        ],
        how="horizontal",
    )
    return features_df

In [ ]:
train_test_df = load_data()

train_features_df = add_kfold(
    fe(train_test_df),
    n_splits=config.N_SPLITS,
    random_state=config.N_SPLITS,
    fold_col=config.FOLD_COL,
)

feature_names = sorted([x for x in train_features_df.columns if x.startswith(config.FEATURE_PREFIX)])
cat_features = [x for x in feature_names if x.startswith(f"{config.FEATURE_PREFIX}c_")]

logger.info(f"# of features: {len(feature_names)}")
logger.info(f"# of cat_features: {len(cat_features)}")


va_result_df_cls_cat, _, _ = single_train_fn(
    model=CatBoostClassifierWrapper(
        name="cat",
        model=CatBoostClassifier(
            loss_function="Logloss",
            learning_rate=0.05,
            n_estimators=10000,
            early_stopping_rounds=500,
            verbose=100,
            # subsample=0.5,
            colsample_bylevel=0.2,
            random_state=config.SEED,
        ),
        multi_output=False,
        feature_names=feature_names,
        cat_features=cat_features,
    ),
    features_df=train_features_df,
    feature_cols=feature_names,
    target_col=config.EVENT_COL,
    fold_col=config.FOLD_COL,
    meta_cols=config.META_COLS,
    out_dir=config.OUTPUT_DIR,
    train_folds=None,
    eval_fn=ROCAUCMetric(),
    overwrite=False,
    use_eval_metric_extra_va_df=False,
)

va_result_df_cls_lgb, _, _ = single_train_fn(
    model=LightGBMWapper(
        name="lgb",
        model=lgb.LGBMModel(
            objective="binary",
            boosting="gbdt",
            n_estimators=10000,
            learning_rate=0.02,
            num_leaves=31,
            colsample_bytree=0.2,
            subsample=0.5,
            importance_type="gain",
        ),
        fit_params={
            "callbacks": [
                lgb.early_stopping(500, first_metric_only=True),
                lgb.log_evaluation(period=100),
            ],
            "categorical_feature": cat_features,
            "feature_name": feature_names,
        },
    ),
    features_df=train_features_df,
    feature_cols=feature_names,
    target_col=config.EVENT_COL,
    fold_col=config.FOLD_COL,
    meta_cols=config.META_COLS,
    out_dir=config.OUTPUT_DIR,
    train_folds=None,
    eval_fn=ROCAUCMetric(),
    overwrite=False,
    use_eval_metric_extra_va_df=False,
)

va_result_df = (
    va_result_df_cls_cat.select([pl.col(config.ID_COL), pl.col("pred").alias("pred_cat")])
    .join(
        va_result_df_cls_lgb.select(config.META_COLS + [pl.col("pred").alias("pred_lgb")]),
        on=config.ID_COL,
        how="inner",
    )
    .with_columns(pl.mean_horizontal(["pred_cat", "pred_lgb"]).alias("pred"))
)
logger.info(f"{ROCAUCMetric()._name}: {ROCAUCMetric()(va_result_df)}")

In [ ]:
train_test_df = load_data()
features_df = load_ensemble_source(train_test_df, config.ENSEMBLE_EXP_NAMES, source_prefix=config.FEATURE_PREFIX)

feature_names = sorted([x for x in features_df.columns if x.startswith(config.FEATURE_PREFIX)])
cat_features = [x for x in feature_names if x.startswith(f"{config.FEATURE_PREFIX}c_")]

logger.info(f"# of features: {len(feature_names)}")
logger.info(f"# of cat_features: {len(cat_features)}")

if "target" not in features_df.columns:
    lower_bound = 0.0
    scaling_factor = va_result_df.select(pl.col("pred").log().min() / (lower_bound - pl.col("target_bfhf").min()))[
        "pred"
    ].to_numpy()[0]
    logger.info(f"scaling_factor: {scaling_factor}")
    _df = va_result_df.select(
        pl.col(config.ID_COL),
        pl.col("pred").alias("sample_weight") + 1,
        pl.when(pl.col(config.EVENT_COL) == 0)
        .then(pl.col("pred").log() / scaling_factor + pl.col("target_bfhf"))
        .otherwise(pl.col("pred").log() / scaling_factor + pl.col("target_bfhf"))
        .alias("target"),
    )
    features_df = features_df.join(_df, on=config.ID_COL, how="left")

# features_df = features_df.drop("target")

In [ ]:
train_features_df = add_kfold(
    features_df,
    n_splits=config.N_SPLITS,
    random_state=config.N_SPLITS,
    fold_col=config.FOLD_COL,
)

va_result_df, va_scores, trained_models = single_train_fn(
    model=LinearWrapper(
        name="ridge",
        model=linear_model.Ridge(
            # power=1.5,
            alpha=50,
            random_state=config.SEED,
        ),
        scaling=False,
        feature_names=feature_names,
    ),
    features_df=train_features_df,
    feature_cols=feature_names,
    target_col="target",
    fold_col=config.FOLD_COL,
    meta_cols=config.META_COLS,
    # weight_col="sample_weight",
    out_dir=config.OUTPUT_DIR,
    train_folds=None,
    eval_fn=Metric(),
    overwrite=True,
    use_eval_metric_extra_va_df=False,
)

In [ ]:
# debug test
test_features_df = load_ensemble_source(
    train_test_df,
    config.ENSEMBLE_EXP_NAMES,
    source_prefix=config.FEATURE_PREFIX,
    output_dataset="TEST",
)

te_result_df = single_inference_fn(
    model=LinearWrapper(
        name="ridge",
        model=linear_model.Ridge(
            alpha=50,
            random_state=config.SEED,
        ),
        scaling=False,
        feature_names=feature_names,
    ),
    features_df=test_features_df,
    feature_names=feature_names,
    model_dir=config.ARTIFACT_EXP_DIR(),
    out_dir=config.OUTPUT_DIR,
    inference_folds=list(range(config.N_SPLITS)),
)
print(te_result_df["pred"].to_list())